In [ ]:
%matplotlib notebook

import math
import sympy

import matplotlib.pyplot as plt

## Trajectoire

Pour le calcul des trajectoires, l'idée est de partir d'un paramètre désiré de commande $x$ et d'élaborer les paramètres lissés qui serviront de référence pour l'asservissement : $y$, $\dot y$ et $\ddot y$ pour une trajectoire d'ordre 2

### Ordre 2

En partant de l'équation canonique d'un filtre d'ordre 2 :

$$ \frac{y}{x} = \frac{1}{1 + 2 \varepsilon \frac{s}{\omega_0} + \frac{s^2}{\omega_0^2}}$$

Avec :

* $\varepsilon$ l'atténuation
* $\omega_0$ la pulsation de coupure

On peut réécrire l'équation comme suit:

$$ x = y \left ( 1 + 2 \varepsilon \frac{s}{\omega_0} + \frac{s^2}{\omega_0^2} \right ) $$

$$ x = y + 2 \varepsilon \frac{s}{\omega_0} y + \frac{s^2}{\omega_0^2} y $$

On remplace les opérateurs en $s$ par les dérivées, ce que l'on cherche c'est à écrire $s^2 y$ ou $\ddot y$, la dérivée seconde de y
$$ x = y + 2 \varepsilon \frac{\dot y}{\omega_0}  + \frac{\ddot y }{\omega_0^2} $$

$$ x - y - 2 \varepsilon \frac{\dot y}{\omega_0} = \frac{\ddot y}{\omega_0^2} $$

$$ \omega_0^2 x - \omega_0^2 y - 2 \varepsilon \omega_0 \dot y = \ddot y $$

$$ \left ( \frac{\omega_0}{2 \varepsilon} x - \frac{\omega_0}{2 \varepsilon} y - \dot y \right ) 2 \omega_0 \varepsilon = \ddot y $$

$$ \left ( \left ( x - y \right ) \frac{\omega_0}{2 \varepsilon} - \dot y \right ) 2 \omega_0 \varepsilon = \ddot y $$

Il ne reste plus qu'à ajouter des limites pour les dérivées premières et secondes :

$$ \left \lfloor \left ( \left \lfloor \left ( x - y \right ) \frac{\omega_0}{2 \varepsilon} \right \rceil - \dot y \right ) 2 \omega_0 \varepsilon \right \rceil = \ddot y $$

Après quoi on peut passer le $\ddot y$ dans un limiteur de variation pour une limite d'ordre 3 pas plus cher

In [ ]:
# check
X, Y, w0, e, s = sympy.symbols('X Y omega_0 epsilon s')
sol = sympy.solve((w0/s)**2 * (X - Y - 2*e/w0*s*Y) - Y, X)
(sol[0] - ( Y * (1 + 2*e*s/w0 + (s/w0)**2))).expand()

Et dessiner :



In [ ]:
class Filter2sat :
    def __init__(self, omega0, epsilon, sat_1, sat_2, period=0.01) :
        self.period = period
        
        self.omega0 = omega0
        self.epsilon = epsilon
        
        # Here saturation is symetric, but it could be implemented to use different values for upper and lower bounds
        self.sat_1 = sat_1
        self.sat_2 = sat_2
                
        self.y0_lst = [0.0,]
        self.y1_lst = [0.0,]
        self.y2_lst = [0.0,]
        
        self.x0_lst = list()
        
        self.n = 0
        self.t_lst = list()
        
    def bound(self, x, lower, upper) :
        return max(lower, min(x, upper))
        
    def run(self, x0) :
        
        # first we compute y2, the second order derivative of the output
        # from the previous values of y0 and y1
        y2 = (x0 - self.y0_lst[-1] - (2 * self.epsilon * self.y1_lst[-1] / self.omega0)) * self.omega0**2
        
        # we apply the sat 2 limitation
        y2 = self.bound(y2, -self.sat_2, self.sat_2)
                
        # y1 is the integration of y2
        y1 = self.y1_lst[-1] + self.period * (self.y2_lst[-1] + y2) / 2
        
        # we apply the sat 1 limitation
        y1 = self.bound(y1, -self.sat_1, self.sat_1)
        
        # y0, the output of the filter is the integration of y1
        y0 = self.y0_lst[-1] + self.period * (self.y1_lst[-1] + y1) / 2
        
        # DEBUG BEGIN
        self.x0_lst.append(x0)
        
        self.y2_lst.append(y2)
        self.y1_lst.append(y1)
        self.y0_lst.append(y0)
        
        self.t_lst.append(self.n * self.period)
        self.n += 1
        # DEBUG END
        
        return y0

In [ ]:
x = [0.0,] * 4 + [12.0,] * 700
u = Filter2sat(1.0, 0.8, 4.0, 8.0)
v = Filter2sat(1.0, 0.8, math.inf, math.inf)

y_sat = [u.run(i) for i in x]
y_flt = [v.run(i) for i in x]

plt.figure()
plt.subplot(3, 1, 1)
plt.plot(v.t_lst, v.y0_lst[1:], label="classic")
plt.plot(v.t_lst, u.y0_lst[1:], label="limited")
plt.plot(v.t_lst, v.x0_lst)
plt.legend()
plt.subplot(3, 1, 2)
plt.plot(v.t_lst, v.y1_lst[1:])
plt.plot(v.t_lst, u.y1_lst[1:])
plt.subplot(3, 1, 3)
plt.plot(v.t_lst, v.y2_lst[1:])
plt.plot(v.t_lst, u.y2_lst[1:])
plt.show()

In [ ]:
class Filter2satWrap :
    def __init__(self, omega0, epsilon, sat_1, sat_2, period=0.02, init=[0.0, 0.0, 0.0]) :
        self.period = period

        self.omega0 = omega0
        self.epsilon = epsilon

        # Here saturation is symetric, but it could be implemented to use different values for upper and lower bounds
        self.sat_1 = sat_1
        self.sat_2 = sat_2

        self.y0_lst = [init[0],]
        self.y1_lst = [init[1],]
        self.y2_lst = [init[2],]

        self.x0_lst = list()

        self.n = 0
        self.t_lst = list()

    def bound(self, x, lower, upper) :
        return max(lower, min(x, upper))

    def run(self, x0) :
        
        ############
        delta = (x0 % 360.0) - self.y0_lst[-1] # x0 is converted in [0; 360)
        if delta > 180.0 :
            delta -= 360.0
        if delta < -180.0 :
            delta += 360.0
            
        # first we compute y2, the second order derivative of the output
        # from the previous values of y0 and y1
        y2 = (delta - (2 * self.epsilon * self.y1_lst[-1] / self.omega0)) * self.omega0**2

        # we apply the sat 2 limitation
        y2 = self.bound(y2, -self.sat_2, self.sat_2)

        # y1 is the integration of y2
        y1 = self.y1_lst[-1] + self.period * (self.y2_lst[-1] + y2) / 2

        # we apply the sat 1 limitation
        y1 = self.bound(y1, -self.sat_1, self.sat_1)

        # y0, the output of the filter is the integration of y1
        y0 = self.y0_lst[-1] + self.period * (self.y1_lst[-1] + y1) / 2
        
        ############
        y0 = (y0 % 360) # output wrapped again in [0;360)

        # DEBUG BEGIN
        self.x0_lst.append(x0)

        self.y2_lst.append(y2)
        self.y1_lst.append(y1)
        self.y0_lst.append(y0)

        self.t_lst.append(self.n * self.period)
        self.n += 1
        # DEBUG END

        return y0

In [ ]:
x = [20,] * 12 + [340.0,] * 200 + [160.0,] * 500
u = Filter2satWrap(1.0, 0.7, 8.0, 16.0, init=[20.0, 0.0, 0.0])

y = [u.run(i) for i in x]

plt.figure()
plt.subplot(3, 1, 1)
plt.plot(u.t_lst, [(i - 360.0 if i > 180.0 else i) for i in u.y0_lst[1:]], label="limited")
plt.plot(u.t_lst, [(i - 360.0 if i > 180.0 else i) for i in u.x0_lst])
plt.grid()
plt.subplot(3, 1, 2)
plt.plot(u.t_lst, u.y1_lst[1:])
plt.grid()
plt.subplot(3, 1, 3)
plt.plot(u.t_lst, u.y2_lst[1:])
plt.grid()
plt.show()

In [ ]:
x = [340,] * 12 + [20.0,] * 200 + [200.0,] * 500
u = Filter2satWrap(1.0, 0.7, 8.0, 16.0, init=[340.0, 0.0, 0.0])

y = [u.run(i) for i in x]

plt.figure()
plt.subplot(3, 1, 1)
plt.plot(u.t_lst, [(i - 360.0 if i > 180.0 else i) for i in u.y0_lst[1:]], label="limited")
plt.plot(u.t_lst, [(i - 360.0 if i > 180.0 else i) for i in u.x0_lst])
plt.grid()
plt.subplot(3, 1, 2)
plt.plot(u.t_lst, u.y1_lst[1:])
plt.grid()
plt.subplot(3, 1, 3)
plt.plot(u.t_lst, u.y2_lst[1:])
plt.grid()
plt.show()